In [1]:
import csv
import os
import pickle
import re
import sys
import numpy as np

from joblib import Parallel, delayed

from difflib import SequenceMatcher,get_close_matches

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D, GlobalMaxPooling1D
from keras.models import Model, Sequential

from numpy import linalg as LA

from scipy import spatial

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from tqdm import *

module_path = os.path.abspath(os.path.join('../helpers/'))
if module_path not in sys.path:
    sys.path.append(module_path)

# my stuff in the helpers/ directory
import embeddings_helper, files_helper, texts_helper, metrics_helper, tags_helper,cobrinha_helper

from cobrinha_helper import evaluate_cobrinha


Using TensorFlow backend.


In [2]:
SEED=np.random.randint(1,1000)
SEED

914

In [3]:
np.random.seed(SEED)

In [4]:
PICKLE_DIR_ROOT = "/media/felipe/SSD_VOLUME/auto-tagger/data/tag-hierarchy/"

In [5]:
texts, labels = files_helper.read_stackoverflow_sample_stanford_tokenized("Medium-Small-Sample-Posts-Shuffled",ssd=True)

In [6]:
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
LABELS_MIN_DOC_COUNT = int(20)
BATCH_SIZE=32
EMBEDDING_DIM=100
NUM_EPOCHS=10
TOKENIZER_FILTERS='' # I will perform tokenization myself

In [7]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters=TOKENIZER_FILTERS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# word => word_index_position
word_index = tokenizer.word_index

# word_index_position => word
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [8]:
truncated_labels = tags_helper.truncate_labels(labels,LABELS_MIN_DOC_COUNT)

In [9]:
lb = preprocessing.MultiLabelBinarizer()
binary_labels = lb.fit_transform(truncated_labels)

In [10]:
# make each document (sequence of word indices) be truncated to 
# MAX_SEQUENCE_LENGTH
tokenized_texts = []

for seq in sequences:
    truncated_seq = seq[:MAX_SEQUENCE_LENGTH]
    tokenized_txt = " ".join([inverse_word_index[idx] for idx in truncated_seq])
    tokenized_texts.append(tokenized_txt)

In [11]:
# extracting IDF weights to weight the embeddings

# snooping
vect = TfidfVectorizer(max_features=MAX_NB_WORDS).fit(tokenized_texts)

feature_names = vect.get_feature_names()
idf = vect.idf_

# word => word IDF
idf_index = dict(zip(vect.get_feature_names(), idf))

In [12]:
document_vectors = vect.transform(tokenized_texts)

In [13]:
NUM_WORDS = document_vectors[0].shape[1]

In [14]:
# document_id => [tag1, tag2, tag3, ...]
document_tag_index = tags_helper.get_document_tag_index(binary_labels,lb.classes_)

# tag => [document_id1, document_id2, document_id3, ...]
tag_document_index = tags_helper.get_tag_document_index(binary_labels, lb.classes_)

### build tag vocabulary

In [15]:
tag_vocabulary = lb.classes_

In [16]:
len(tag_vocabulary)

1704

In [17]:
pickle.dump(tag_vocabulary,open(PICKLE_DIR_ROOT+"/tag_vocabulary.p","wb"))

In [18]:
i = 2
tokenized_texts[i],document_tag_index[i]

("use with gulp . i read about gulp and was quite taken by the . i want to try it out for myself but i am running into a little problem . i am used to using with grunt and i have no idea how to get to play nice with gulp . i 've come across this article which suggests there is no need for a plugin when using gulp . unfortunately it does n't really explain how to go about it and the example it links to does n't help me much . is there anyone who knows how to go about this ?",
 ['gulp', 'jekyll'])

### build tag_vectors_index

> if a tag is the ONLY tag assigned to some document, that's probably a more representative document of this tag, than if this tag were only one among many others


In [19]:
# tag => tag_vector
tag_vectors_index = dict()

for (tag,document_ids) in tag_document_index.items():
    
    tag_vector = np.zeros(NUM_WORDS)
    
    for document_id in document_ids:
        document_vector = document_vectors[document_id]
        num_tags = len(document_tag_index[document_id])


        weight = 1.0 / num_tags

        weighted_document_vector = document_vector * weight
        
        tag_vector = tag_vector + weighted_document_vector
        
    tag_vectors_index[tag] = np.asarray(tag_vector).ravel()

In [20]:
pickle.dump(tag_vectors_index,open(PICKLE_DIR_ROOT+"/tag_vectors_index.p","wb"))

In [21]:
normalized_tag_vectors_index = dict()    

## THIS SUCKS.. WHY?
## normalize vectors to make cosine similarity more accurate  
for (tag,tag_vector) in tag_vectors_index.items():    
    normalized_tag_vectors_index[tag] = LA.norm(tag_vector, 2)

In [22]:
def _cosine_similarity(a,b):
    return 1 - spatial.distance.cosine(a, b)

In [23]:
target_tags = ['sql-server-2008','sql','sql-server','mysql','database','postgresql','select','join','oracle','tsql','sqlite']

pairs = [ (a,b) for a in target_tags for b in target_tags if a != b ]

In [24]:
for (tag_a,tag_b) in pairs:
    tag_a_vector = tag_vectors_index[tag_a]
    tag_b_vector = tag_vectors_index[tag_b]
    
    sim = _cosine_similarity(tag_a_vector,tag_b_vector)
    
    print("SIM '{}' '{}' => {}".format(tag_a,tag_b,sim))
    

SIM 'sql-server-2008' 'sql' => 0.9349813624590992
SIM 'sql-server-2008' 'sql-server' => 0.9660928085751612
SIM 'sql-server-2008' 'mysql' => 0.815457038141873
SIM 'sql-server-2008' 'database' => 0.81560949073031
SIM 'sql-server-2008' 'postgresql' => 0.7553913999451284
SIM 'sql-server-2008' 'select' => 0.7779014439193458
SIM 'sql-server-2008' 'join' => 0.7190542723828269
SIM 'sql-server-2008' 'oracle' => 0.7645263559863348
SIM 'sql-server-2008' 'tsql' => 0.9261995236130321
SIM 'sql-server-2008' 'sqlite' => 0.6563332228211394
SIM 'sql' 'sql-server-2008' => 0.9349813624590992
SIM 'sql' 'sql-server' => 0.9460801230737476
SIM 'sql' 'mysql' => 0.9066462947865959
SIM 'sql' 'database' => 0.8520096718309694
SIM 'sql' 'postgresql' => 0.8138906015279469
SIM 'sql' 'select' => 0.8654416823443406
SIM 'sql' 'join' => 0.8200093703093696
SIM 'sql' 'oracle' => 0.8284712719791275
SIM 'sql' 'tsql' => 0.9364441178014254
SIM 'sql' 'sqlite' => 0.7008765295592592
SIM 'sql-server' 'sql-server-2008' => 0.9660928

if all other tags are, on average, more similar to A than to B, then A is probably more generic than B

should it be normalized by the difference between A and B?

will fail for tags that are below B?

In [25]:
sql_vector = tag_vectors_index['sql']
avg_diff = list()

for (tag_name,tag_vector) in tag_vectors_index.items():
    avg_diff.append(_cosine_similarity(tag_a_vector,sql_vector))
    
np.mean(np.array(avg_diff))

0.70087652955925939

In [26]:
sql_vector = tag_vectors_index['sql-server']
avg_diff = list()

for (tag_name,tag_vector) in tag_vectors_index.items():
    avg_diff.append(_cosine_similarity(tag_a_vector,sql_vector))
    
np.mean(np.array(avg_diff))

0.66694055592038359

In [27]:
sql_vector = tag_vectors_index['sql-server-2008']
avg_diff = list()

for (tag_name,tag_vector) in tag_vectors_index.items():
    avg_diff.append(_cosine_similarity(tag_a_vector,sql_vector))
    
np.mean(np.array(avg_diff))

0.65633322282113937

In [28]:
def get_sim(tag_a,tag_b):
    
    sim = _cosine_similarity(tag_vectors_index[tag_a],tag_vectors_index[tag_b])
    
    return (tag_a,tag_b,sim)

In [29]:
all_tags = [tag for (tag,_) in tag_vectors_index.items()]

In [30]:
sims = Parallel(n_jobs=-1)(delayed(get_sim)(a,b) for a in all_tags for b in all_tags)

In [31]:
sims[:10]

[('beautifulsoup', 'beautifulsoup', 1.0),
 ('beautifulsoup', 'gruntjs', 0.18613827549594275),
 ('beautifulsoup', 'jpa', 0.27220107789865933),
 ('beautifulsoup', 'artificial-intelligence', 0.28545351845925149),
 ('beautifulsoup', 'ip-address', 0.16893513341351518),
 ('beautifulsoup', 'overlay', 0.21743887537964235),
 ('beautifulsoup', 'windows-phone-8', 0.298729857708814),
 ('beautifulsoup', 'interop', 0.27860879445993858),
 ('beautifulsoup', 'heap', 0.196834828791418),
 ('beautifulsoup', 'httpwebrequest', 0.32254657278229204)]

In [32]:
similarity_dict = dict()

for tag_a,tag_b, sim in sims:
    
    if similarity_dict.get(tag_a) is None:
        similarity_dict[tag_a] = [(tag_b,sim)]
    else:
        similarity_dict[tag_a].append((tag_b,sim))

In [33]:
similarity_dict["sql"][:20]

[('beautifulsoup', 0.35604607295605595),
 ('gruntjs', 0.26506919039400378),
 ('jpa', 0.58291538943726939),
 ('artificial-intelligence', 0.4530957555900843),
 ('ip-address', 0.25310359189492693),
 ('overlay', 0.29155430241955815),
 ('windows-phone-8', 0.46598419316765083),
 ('interop', 0.44678801385464184),
 ('heap', 0.31570016256364952),
 ('httpwebrequest', 0.39770535712161581),
 ('windows-xp', 0.38805061065445923),
 ('signal-processing', 0.47842361178174997),
 ('master-pages', 0.29224029307961652),
 ('time-series', 0.56772556594281853),
 ('visual-c++', 0.57092830032009423),
 ('hive', 0.39653067080753901),
 ('localization', 0.55144557086492441),
 ('config', 0.43292960593197916),
 ('constructor', 0.42352721206183941),
 ('posix', 0.49470383818174102)]

In [34]:
sorted(similarity_dict["sql"],key=lambda tpl : tpl[1],reverse=True)[1:20]

[('sql-server', 0.94608012307374756),
 ('tsql', 0.93644411780142545),
 ('sql-server-2008', 0.93498136245909924),
 ('mysql', 0.90664629478659586),
 ('select', 0.8654416823443406),
 ('sql-server-2005', 0.85241314058735329),
 ('database', 0.85200967183096943),
 ('sql-server-2008-r2', 0.84317564199824524),
 ('oracle', 0.82847127197912751),
 ('sql-server-2012', 0.82132152550983883),
 ('join', 0.8200093703093696),
 ('postgresql', 0.8138906015279469),
 ('ms-access', 0.80220855469972741),
 ('database-design', 0.78300579086276745),
 ('sql-update', 0.77638548602713653),
 ('ms-access-2010', 0.77491125431556873),
 ('group-by', 0.77284000412203369),
 ('subquery', 0.76447204415239334),
 ('query-optimization', 0.74736265127425294)]

In [35]:
sorted_similarity_dict = dict()

for (tag, similarities_to_other_tags) in similarity_dict.items():
    
    sorted_similarities = sorted(similarities_to_other_tags,key=lambda tpl: tpl[1],reverse=True)
    sorted_without_self = sorted_similarities[1:]
    
    sorted_similarity_dict[tag] = sorted_without_self

In [36]:
sorted_similarity_dict["sql"][:20]

[('sql-server', 0.94608012307374756),
 ('tsql', 0.93644411780142545),
 ('sql-server-2008', 0.93498136245909924),
 ('mysql', 0.90664629478659586),
 ('select', 0.8654416823443406),
 ('sql-server-2005', 0.85241314058735329),
 ('database', 0.85200967183096943),
 ('sql-server-2008-r2', 0.84317564199824524),
 ('oracle', 0.82847127197912751),
 ('sql-server-2012', 0.82132152550983883),
 ('join', 0.8200093703093696),
 ('postgresql', 0.8138906015279469),
 ('ms-access', 0.80220855469972741),
 ('database-design', 0.78300579086276745),
 ('sql-update', 0.77638548602713653),
 ('ms-access-2010', 0.77491125431556873),
 ('group-by', 0.77284000412203369),
 ('subquery', 0.76447204415239334),
 ('query-optimization', 0.74736265127425294),
 ('oracle-sqldeveloper', 0.74190041768960613)]

In [37]:
pickle.dump(sorted_similarity_dict,open(PICKLE_DIR_ROOT+"/sorted_similarity_dict.p",'wb'))

### build global similarity index

In [38]:
global_similarity_index = dict()

for tag in tag_vocabulary:
    
    similarities_with_current_tag = list()
    
    for other_tag, similarity_to_other_tag in sorted_similarity_dict[tag]:
        
        if other_tag != tag:
            similarities_with_current_tag.append(similarity_to_other_tag)

    global_avg_sim_wrt_tag = np.array(similarities_with_current_tag).mean()
    
    global_similarity_index[tag] = global_avg_sim_wrt_tag
            
pickle.dump(global_similarity_index,open(PICKLE_DIR_ROOT+"/global_similarity_index.p",'wb'))

In [39]:
# global_similarity_index

In [40]:
sorted(global_similarity_index.items(),key=lambda tpl: tpl[1],reverse=True)[:20]

[('c#', 0.52796384399674001),
 ('.net', 0.52237488943626997),
 ('web', 0.49698595034123894),
 ('c#-4.0', 0.49237870705839931),
 ('performance', 0.49045843519413501),
 ('design', 0.48792240596086045),
 ('java', 0.48656225358421379),
 ('javascript', 0.4859904125176337),
 ('asp.net', 0.48341469702671458),
 ('security', 0.48225525430704314),
 ('php', 0.47973539893855099),
 ('user-interface', 0.47965186513161667),
 ('.net-4.0', 0.47752612700700081),
 ('html5', 0.47723670445625721),
 ('optimization', 0.47654262078466109),
 ('windows', 0.47547176112432721),
 ('ios', 0.47507086838459761),
 ('cocoa', 0.47291592025162482),
 ('cocoa-touch', 0.47099675887289727),
 ('winforms', 0.46932929711010801)]

In [41]:
evaluate_cobrinha('android','android-service',tag_vectors_index,sorted_similarity_dict, global_similarity_index) 

(0.13861274631348885, 0.68468875828253828)

In [42]:
evaluate_cobrinha('asp.net','asp.net-mvc',tag_vectors_index,sorted_similarity_dict, global_similarity_index) 

(0.034241093906521636, 0.84005391327352352)

In [43]:
evaluate_cobrinha('asp.net-mvc','asp.net-mvc-5',tag_vectors_index,sorted_similarity_dict, global_similarity_index) 

(0.01188788746170899, 0.89472933091423534)

what about unrelated stuff

In [44]:
evaluate_cobrinha('java','arrays',tag_vectors_index,sorted_similarity_dict, global_similarity_index) 

(0.1155915868012774, 0.62033156246617382)

In [45]:
evaluate_cobrinha('ruby','python-2.7',tag_vectors_index,sorted_similarity_dict, global_similarity_index) 

(-0.01237052618558937, 0.64212738224793176)

In [46]:
evaluate_cobrinha('database','python-2.7',tag_vectors_index,sorted_similarity_dict, global_similarity_index) 

(0.0202516829136134, 0.64969036563896276)

what's the mean and stddev of the similarity between all tags and each other?

In [47]:
running_avgs = []

for (tag, similarities_to_other_tags) in similarity_dict.items():
    sims = [tpl[1] for tpl in similarities_to_other_tags]
    
    avg = np.array(sims).mean()
    
    running_avgs.append(avg)
    
np.array(running_avgs).mean(),np.array(running_avgs).std()

(0.31304959489683454, 0.064950279174516554)